# TFL Bike data Analysis

#### Exploratory data analysis as part of my MSc thesis, "Using machine learning to predict Transport for London bike sharing habits in the post COVID-19 era".

The following code for downloading the data has been adopted from [Markus Hauru's](https://github.com/mhauru) analysis, 'Predicting Boris Bike usage'.



In [24]:
# importing libraries

import os
import pickle
import requests
import zipfile
import pandas as pd
import numpy as np
import scipy as sp
import statsmodels.api as sm
from sklearn import linear_model, svm, neighbors, tree
from matplotlib import pyplot as plt
import matplotlib
import seaborn as sns
from pathlib import Path
from timeit import default_timer as timer
from IPython.display import set_matplotlib_formats
from urllib.parse import urlparse

try:
    import xlrd
except Exception as e:
    msg = (
        "Please install the package xlrd: `pip install --user xlrd`"
        "It's an optional requirement for pandas, and we'll be needing it."
    )
    print(msg)
    raise e

In [25]:
# For pretty and exportable matplotlib plots.
# If you are running this yourself and want interactivity,
# try `%matplotlib widget` instead.
set_matplotlib_formats("svg")
%matplotlib inline
# %matplotlib widget
# Set a consistent plotting style across the notebook using Seaborn.
sns.set_style("darkgrid")
sns.set_context("notebook")
# Make pandas cooperate with pyplot
pd.plotting.register_matplotlib_converters()


C:\Users\EMoses\AppData\Local\Temp\ipykernel_10704\3312271832.py:4: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats("svg")


1. Processing and cleaning the bike data
Before getting anywhere with it, we'll need to process the bike data quite a bit. The data comes in CSV files, each of which covers a period of time. Up first, we need to download the data from the TfL website. If you are running this code yourself, here's a script that does that. Be warned though, it's almost seven gigs of data. You can run it repeatedly, and it'll only download data that it doesn't have already.

In [26]:
bikefolder = "data/bikes"

In [27]:
def download_file(datafolder, url, verbosity=0):
    """Download the data from the given URL into the datafolder, unless it's
    already there. Return path to downloaded file.
    """
    datafolder = Path(datafolder)
    datafolder.mkdir(parents=True, exist_ok=True)

    a = urlparse(url)
    filename = Path(os.path.basename(a.path))
    filepath = datafolder / filename
    # Don't redownload if we already have this file.
    if filepath.exists():
        if verbosity > 1:
            print("Already have {}".format(filename))
    else:
        if verbosity > 0:
            print("Downloading {}".format(filename))
        rqst = requests.get(url)
        rqst.raise_for_status()
        with open(filepath, "wb") as f:
            f.write(rqst.content)
    return filepath


In [30]:
# Adjust whether to print progress reports of the downloads.
# verbosity=0 is silence, verbosity=1 reports only when actually doing things,
# verbosity>1 also reports when there's nothing to do.
verbosity = 1

# Most files are individual CSV files, listed in bike_data_urls.txt. Download
# them.
urlsfile = "data/bikes/bike_data_urls.txt"
with open(urlsfile, "r") as f:
    urls = f.read().splitlines()
# There are a few comments in the file, marked by lines starting with #.
# Filter them out.
urls = [u for u in urls if u[0] != "#"]
for url in urls:
    download_file(bikefolder, url, verbosity)

# The early years come in zips. Download and unzip them.
zipsfolder = Path("data/bikes/bikezips")
bikezipurls = [
    "https://cycling.data.tfl.gov.uk/usage-stats/cyclehireusagestats-2012.zip",
    "https://cycling.data.tfl.gov.uk/usage-stats/cyclehireusagestats-2013.zip",
    "https://cycling.data.tfl.gov.uk/usage-stats/cyclehireusagestats-2014.zip",
    "https://cycling.data.tfl.gov.uk/usage-stats/2015TripDatazip.zip",
    "https://cycling.data.tfl.gov.uk/usage-stats/2016TripDataZip.zip",
]
# A list of CSV files that are already there. Only unzip if some of the files
# in the zip aren't present already.
current_csvs = sorted(os.listdir(bikefolder))
for url in bikezipurls:
    zippath = download_file(zipsfolder, url, verbosity)
    with zipfile.ZipFile(zippath, "r") as z:
        namelist = z.namelist()
        has_been_extracted = any(name not in current_csvs for name in namelist)
        if has_been_extracted:
            if verbosity > 0:
                print("Unzipping {}".format(zippath))
            z.extractall(bikefolder)
        else:
            if verbosity > 1:
                print("{} has already been extracted.".format(zippath))

# Finally, there's an odd one out: One week's data comes in as an .xlsx.
# Download it and use pandas to convert it to csv.
xlsxurl = "https://cycling.data.tfl.gov.uk/usage-stats/49JourneyDataExtract15Mar2017-21Mar2017.xlsx"
xlsxfile = download_file(bikefolder, xlsxurl)
csvfile = xlsxfile.with_suffix(".csv")
if not csvfile.exists():
    if verbosity > 0:
        print("Converting .xlsx to .csv.")
    pd.read_excel(xlsxfile).to_csv(csvfile, date_format="%d/%m/%Y %H:%M:%S")
else:
    if verbosity > 1:
        print("Already have {}".format(csvfile))

Converting .xlsx to .csv.


The data we have now lists on each line of the CSV file a single bike trip, with starting point and time, end point and time, and things like bike ID number. Here's an example.

In [31]:
example_file  = Path(bikefolder) / Path("47JourneyDataExtract01Mar2017-07Mar2017.csv")
pd.read_csv(example_file, encoding="ISO-8859-2").head()

,Rental Id,Duration,Bike Id,End Date,EndStation Id,EndStation Name,Start Date,StartStation Id,StartStation Name
0,62857677,3780.0,7851,06/03/2017 19:20,43.0,"Crawford Street, Marylebone",06/03/2017 18:17,811,"Westferry Circus, Canary Wharf"
1,62863035,540.0,4089,06/03/2017 22:17,295.0,"Swan Street, The Borough",06/03/2017 22:08,272,"Baylis Road, Waterloo"
2,62775896,600.0,4895,02/03/2017 21:27,295.0,"Swan Street, The Borough",02/03/2017 21:17,197,"Stamford Street, South Bank"
3,62747748,420.0,4347,01/03/2017 21:08,295.0,"Swan Street, The Borough",01/03/2017 21:01,803,"Southwark Street, Bankside"
4,62843939,420.0,3192,06/03/2017 09:28,193.0,"Bankside Mix, Bankside",06/03/2017 09:21,197,"Stamford Street, South Bank"


In [37]:
test_df = pd.read_csv(example_file, encoding="ISO-8859-2")

In [38]:
test_df.shape

(163110, 9)

In [39]:
test_df['StartStation Name'].value_counts()

Belgrove Street , King's Cross          1834
Waterloo Station 3, Waterloo            1744
Waterloo Station 1, Waterloo            1052
Hyde Park Corner, Hyde Park              881
Finsbury Circus, Liverpool Street        822
                                        ... 
Oxford Road, Putney                       22
Thornfield House, Poplar                  18
Grant Road Central, Clapham Junction      13
Castalia Square, Cubitt Town               8
Mechanical Workshop Penton                 2
Name: StartStation Name, Length: 778, dtype: int64

In [40]:
test_df['EndStation Name'].value_counts()

Belgrove Street , King's Cross       1838
Waterloo Station 3, Waterloo         1590
Hop Exchange, The Borough            1116
Waterloo Station 1, Waterloo         1041
Finsbury Circus, Liverpool Street     909
                                     ... 
South Quay East, Canary Wharf          18
Aberfeldy Street, Poplar               17
St. John's Park, Cubitt Town           16
Thornfield House, Poplar               15
Mechanical Workshop Penton              5
Name: EndStation Name, Length: 778, dtype: int64